In [23]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy.sparse import csc_matrix
from scipy.sparse.linalg import svds


from google.cloud import bigquery

In [5]:
project_id = 'listenbrainzrecommender'
client = bigquery.Client(project=project_id)

In [3]:
# Authenticate google cloud platform
# from google.colab import auth
# auth.authenticate_user()
# print('Authenticated')

Authenticated


# FSM

So far we've gotten the songs by the total amount of plays. We'll continue to do this for the top 25 songs but it's important to know that a few of the top songs have very few listeners.

This should become less of an issue as we add more songs in iterative models.

For the FSM, we'll just work with the top 25 songs (by listens) for starts.

In [8]:
# get top 25 songs by play amount
top25songs = client.query('''
SELECT
  COUNT(user_name) AS song_plays,
  artist_name AS artist,
  track_name AS title

FROM
  `listenbrainz.listenbrainz.listen`
GROUP BY
  artist_name, track_name
ORDER BY
  song_plays DESC
LIMIT
  25
''').to_dataframe()

In [9]:
top25songs.head()

,song_plays,artist,title
0,113348,近藤浩治,Inside the Deku Tree
1,49199,Michael Z. Land,Docks Background Ambience
2,15159,Nirvana,Smells Like Teen Spirit
3,14378,Radiohead,Karma Police
4,13346,Massive Attack,Teardrop


In [15]:
# v2 song class
class Song:
  def __init__(self, artist_name, song_name):
    self.artist_name = artist_name
    self.song_name = song_name
  
  def get_listener_list(self, request):
    '''
    Gets the listeners for a given song.
    Does not take into account the amount an account has listened to a song.
    Returns a dataframe if requested.
    '''
    self.listeners = client.query('''
      SELECT
        user_name
      FROM
        `listenbrainz.listenbrainz.listen`
      WHERE
        artist_name = \''''
      + self.artist_name + '''\'
        AND
        track_name = \''''
      + self.song_name + '''\'
      GROUP BY
        user_name
      ''').to_dataframe()

    if request:
      return self.listeners

In [16]:
# Create song objects for top 25 songs
listOfSongs= [(Song(row.artist,row.title)) for index, row in top25songs.iterrows() ]

In [17]:
listOfSongs[3].song_name

'Karma Police'

In [19]:
# get listeners for listOfSongs
for x in listOfSongs:
  x.get_listener_list(request=False)

# Timing: 30s to get 25 songs

In [20]:
listOfSongs[3].listeners.head()

,user_name
0,Xenatier
1,ruanmuller
2,Polarimetric
3,iliekcomputers
4,deathwarmedover


In [22]:
# sparse_plays = pd.DataFrame()

In [ ]:
song_df = pd.DataFrame()